# Μεθόδος Επίλυσης Μη-Γραμμικού Συστήματος 2 Μεταβλητών με τη βοήθεια της Διαστηματικής Newton
### Παντελής Λιναρδάτος (13182) & Γιώργος Παπασωτηρόπουλος (13249)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from interval import interval, inf, imath

Για αρχή θυμίζουμε ότι κάθε διάστημα x θα δίνεται ως x=interval[a,b] ενώ για να πάρουμε τους πραγματικούς αριθμούς a και b θα χρησιμοποιούμε τις εντολές x[0].inf και x[0].sup αντίστοιχα. 

Επίσης κάθε συνάρτηση f(x) συμπεριφέρεται ως πραγματική ή διαστηματική συνάρτηση ανάλογα με το όρισμα που θα παρει, εάν δηλαδή το x είναι διάστημα (ή εκφυλισμένο διάστημα) θα πάρουμε διαστηματικό υπολογισμό, ενώ εάν το x είναι πραγματικός αριθμός θα πάρουμε την τιμή της f εκεί.

#### Η Συνάρτηση του Πλάτους ενός Διαστήματος:


In [2]:
def width(x): 
    return float(x[0].sup-x[0].inf)

####  Ο Ορισμός των Προς Χρήση Συναρτήσεων δύο μεταβλητών:


In [3]:
def f1(x,y):
    return x*y

def f2(x,y):
    return x**2+y**2

####  Ο Ορισμός των προς Χρήση Παραγώγων (ως προς y γιατι το x το κρατάμε συνεχώς σταθερό):

In [4]:
def df1(x,y):
    return x

def df2(x,y):
    return 2*y

#### Ο Ορισμός της Συνάρτησης του Τελεστή Newton για τις f1,f2:
Αυτός ισούται με τη διαφορά του μέσου του δεδομένου διαστήματος (y, αφού το x είναι σταθερό) μείον το πηλίκο της συνάρτησης στο μέσο προς την παράγωγο στο διάστημα. 
Θα έχουμε mid το μέσο, ar τον αριθμητή του κλάσματος (εκφυλισμένο διάστημα της συνάρτησης στο (x,μέσο)).

In [5]:
def NewtonStep(x,y,F1,dF1):               
    mid=y.midpoint
    ar=interval[F1(x,mid),F1(x,mid)]
    return mid-ar/dF1(x,y)

In [6]:
def NewtonStep2(x,y,F2,dF2):               
    mid=y.midpoint
    ar=interval[F2(x,mid),dF2(x,mid)]
    return mid-ar/dF2(x,y)

#### Η Συνάρτηση Interval-Newton για δύο μεταβλητές σταθεροποιώντας την πρώτη (x):

In [7]:
def Newton_xstath(x,y,F,dF,tol,maxk): #ορίσματα: δύο διαστήματα, συνάρτηση και παράγωγος της, ακρίβεια
    #και μέγιστο πλήθος επιτρεπόμενων επαναλήψεων 
    ylist=[] #λίστα των διαστημάτων προς εξέταση
    ylist.append(y) #αρχικοποίηση
    Q=[] #λίστα των διαστημάτων στα οποία υπάρχει ρίζα
    k=0 #μετρητής πλήθους επαναλήψεων       
    while ylist and maxk>=k: #όσο η λίστα δεν είναι κενή και το κριτήριο επαναλήψεων δεν ικανοποιείται:
        k=k+1 #αύξηση μετρητή πλήθους επαναλήψεων
        y=ylist.pop() #επιλογή του πρώτου στοιχείου για το y της λίστας του
        Fdiast=F(x,y) #διαστηματικός υπολογισμός της f και της παραγώγου της στα x(σταθερό),y(το 1ο της λίστας) 
        dFdiast=dF(x,y)
        if 0 in Fdiast: #εάν το 0 ανήκει στον διαστ. υπολογισμό
            if width(y)<tol: #και εάν ικανοποιεί το κριτήριο πλάτους
                Q.append(y) #μπαίνει στη λίστα των πιθανών ριζών
            elif 0 in dFdiast: #αν δεν ικανοποιεί το κριτήριο πλάτους αλλά το 0 ανήκει στον διαστ.υπολ. παραγώγου
                mid=y.midpoint[0].inf #διχοτομείται το y σε δύο υποδιαστήματα που μπαίνουν στη λίστα για έλεγχο
                y1=interval[y[0].inf,mid]
                y2=interval[mid,y[0].sup]
                ylist.append(y1)
                ylist.append(y2)
            else:   #αλλιώς (0 δεν ανήκει στον διαστ.υπολ. της παραγώγου)
                    #εφαρμόζεται ο τελεστής Newton (που ορίζεται) και η τομή του με το y (αν υπάρχει)
                    #μπαίνει στη λίστα
                ynew=NewtonStep(x,y,F,dF)
                if ynew&y:
                    ylist.append(ynew&y)
    return Q #επιστρέφεται η λίστα Q μόλις επιτευχθεί κάποιο κριτήριο τερματισμού.

#### Εφαρμογή:

In [8]:
x1=interval[-5,5] #Αυτό είναι που κρατάμε σταθερό.
x2=interval[-10,10] #Αυτό είναι το διάστημα αναζήτησης για το x2.
#η λύση του συστηματος είναι το (0,0)
tol=0.01 #ορίζουμε τη ζητούμενη ακρίβεια
kapa=10000 #και το μεγιστο πλήθος επαναληψεων
t1=Newton_xstath(x1,x2,f1,df1,tol,kapa) #τρέξιμο συνάρτησης για την f1
t2=Newton_xstath(x1,x2,f2,df2,tol,kapa) #ομοια για την f2
print "η Newton για την πρώτη συνάρτηση επιστρέφει %d διαστήματα ενώ για την δεύτερη επιστρέφει %d διαστήματα" %(len(t1),len(t2)) #πλήθος επιστρεφόμενων διαστημάτων
print "η λύση του συστήματος είναι στο διάστημα", list(set(t1) & set(t2)) #τομή των διαστημάτων

η Newton για την πρώτη συνάρτηση επιστρέφει 2048 διαστήματα ενώ για την δεύτερη επιστρέφει 2 διαστήματα
η λύση του συστήματος είναι στο διάστημα [interval([0.0, 0.009765625]), interval([-0.009765625, 0.0])]


Προβληματισμοί: 

α) Εάν η τομή των διαστημάτων που οι Newton επέστρεψαν έχει περισσότερα από ένα σύνολα, ποιο κρατάω για να προχωρήσω σε επόμενη επανάληψη της μεθόδου;

β) Αφού η Newton βρίσκει το ελάχιστο δυνατό διάστημα, έστω x2new, που περικλείει κάποια ρίζα, τι νόημα έχει να ξανατρέξω τη Newton για το x2new στην επόμενη επανάληψη της μεθόδου;